# Analizador de sentimientos para comentarios de películas utilizando DNN

## (Reto 01) Paso 1  - Lectura del conjunto de información

In [ ]:
# Importar todas las librerías que se utilizarán
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, Conv1D, LSTM, GlobalMaxPooling1D
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt

import spacy
pln_es = _____._____("es_core_news_lg")

In [ ]:
# Leemos el archivo y mostramos los primeros elementos para verificar
# que sean los correctos
movie_reviews = pd._____("Peliculas_Reseñas_10000.csv")
# Verificar que no existan valore nulos
print("Existen valores nulos:")
print(movie_reviews._____().values.any())
movie_reviews.head(10)

In [ ]:
# Imprimimos uno de los ejemplos para revisar su escritura
movie_reviews["reseña"][300]

In [ ]:
# Ver la distribución que tenemos de los datos
Pos = 0
Neg = 0

for Sentiment in movie_reviews["sentimiento"]:
    if(_____ == 'positive'):
        Pos += 1
    else:
        Neg += 1

plt.bar(['Positivas', 'Negativas'], [Pos, Neg], color = ['lime', 'red'])

## (Reto 02) Paso 2 - Preprocesamiento de la información

In [ ]:
# Filtrado de StopWords utilizando NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))

X = []

removedor_tags = re.compile(r'<[^>]+>')

sentences = list(movie_reviews['reseña'])
for sen in sentences:

    # Filtrado de stopword
    for stopword in stop_words:
        sentence = sen.replace(" " + _____ + " ", " ")

    # Remover los elementos de HTML (Que aparecen en los comentarios)
    sentence = removedor_tags.sub('', sentence)
    # Remover espacios múltiples
    sentence = re.sub(r'\s+', ' ', sentence)
    # Convertir todo a minúsculas
    sentence = sentence._____()
    # Filtrado de signos de puntuación
    tokenizer = RegexpTokenizer(r'\w+')
    # Tokenización del resultado (Aplicando el rechazo de tokens descrito)
    result = tokenizer.tokenize(_____)
    # Agregar al arreglo los textos "destokenizados" (Como texto nuevamente)
    X.append(TreebankWordDetokenizer().detokenize(result))

print(X[:3])

In [ ]:
# Filtrado de más StopWords (Definidas por el usuario)
New_StopWords = ['a','acá','ahí','al','algo','algún','alguna','alguno','algunas','algunos','allá','allí','ambos','ante',
                 'antes','aquel','aquella','aquello','aquellas','aquellos','aquí','arriba','así','atrás','aun','aunque',
                 'bien','cada','casi','como','con','cual','cuales','cualquier','cualquiera','cuan','cuando','cuanto','cuanta',
                 'cuantos','cuantas','de','del','demás','desde','donde','dos','el','él','ella','ello','ellas','ellos','en',
                 'eres','esa','ese','eso','esas','esos','esta','esto','estas','estos','este','etc','ha','hasta','la','lo','las',
                 'los','me','mi','mis','mía','mías','mío','míos','mientras','muy','ni','nosotras','nosotros','nuestra',
                 'nuestro','nuestras','nuestros','os','otra','otro','otras','otros','para','pero','pues','que','qué','si','sí',
                 'siempre','siendo','sin','sino','so','sobre','sr','sra','sres','sta','su','sus','te','tu','tus','un','una',
                 'uno','unas','unos','usted','ustedes','vosotras','vosotros','vuestra','vuestro','vuestras','vuestros','y','ya',
                 'yo']


X_clear = []
# Lematizamos el texto y filtramos las nuevas StopWords
for texto in X:
    spacy_texto = pln_es(texto)
    lemmas = [tok._____.lower() for tok in spacy_texto if not tok.text in _____]
    X_clear.append(' '.join(_____))

print("Textos preprocesados:")
print(X_clear[:3])

In [ ]:
# Creación del vector objetivo (Postivo / Negativo)
Sent = movie_reviews['sentimiento']

y = [1 if _____ == "positive" else 0 for sentimiento in Sent]

print("Vector de objetivos:")
print(y)

In [ ]:
# Separación de la información en conjuntos de entrenamiento
X_train, X_test, y_train, y_test = _____(np.array(X_clear), np.array(y), test_size=0.20)

# Preparamos la capa de embeddingsn(Predefinimos una cantidad de
# 5000 palabras consideradas como tokens
# SECUENCIAS DE TEXTO
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(_____)

# Transforma cada texto en una secuencia de valores enteros
X_train = tokenizer._____(X_train)
X_test = tokenizer._____(X_test)

print("Ejemplo de vectores de entrada:")
print(X_train[:3])
print("\nEjemplo de salidas")
print(y_train[:3])

In [ ]:
# Conversión de los conjuntos de entrenamiento a vectores
maxlen = 100

# Aquellos mayores a 100 son truncados, y los menores les
# es replicado su último valor para tener arreglos
# bidimensionales del mismo tamaño
# A este proceso se le conoce como PADDING
X_train = _____(X_train, padding='post', maxlen=maxlen)
X_test = _____(X_test, padding='post', maxlen=maxlen)

print("Ejemplos para el entrenamiento")
print(len(X))
print("\nCantidad de elementos en vector:")
print(len(X_train[0]))

## Paso 03 -Verificar la coherencia de la información

In [ ]:
# Verificar que la información de entrada
print(X_train)
print(y_train)

## (Reto 3) Pasos 4 y 5 - Selección y aplicación del modelo de IA (DNN)

In [ ]:
# Declaración de librerías para manejo de arreglos (Numpy)
from numpy import asarray
from numpy import zeros

# Lectura del archivo de embeddings
embeddings_dictionary = dict()
Embeddings_file = open('Word2Vect_Spanish.txt', encoding="utf8")

# Extraemos las características del archivo de embeddings
# y las agregamos a un diccionario (Cada elemento es un vextor)
for linea in Embeddings_file:
    caracts = linea.split()
    palabra = caracts[0]
    vector = asarray(caracts[1:], dtype='float32')
    embeddings_dictionary [palabra] = vector
Embeddings_file.close()

In [ ]:
# Extraemos la cantidad de palabras en el vocabulario
vocab_size = len(tokenizer.word_index) + 1

# Generamos la matriz de embeddings (De entrada)
embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    # Extraemos el vector de embedding para cada palabra
    embedding_vector = embeddings_dictionary.get(word)
    # Si la palbra si existía en el vocabulario
    # agregamos su vector de embeddings en la matriz
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
# Definimos las capas de nuestro modelo neuronal (DNN)
# Definimos que el modelo será secuencial (Y que ingresaremos cada capa en el orden de configuración de la red)
model = Sequential()

# Definimos los valores para nuestra capa de Embeddings
embedding_layer = _____(vocab_size, 300, weights=[_____], input_length=maxlen, trainable=False)

# Agregamos al modelo 3 capas (La de Embeddings, una de aplanamiento y una Profunda con una Dense con una salida
# y una activación Sigmoindal)
model.add(_____)
model.add(Flatten())
model.add(Dense(_____, activation='sigmoid'))

# Definimos métodos de optimización, que métrica se utilizará y como calcular la pérdida
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Para ver la configuración que definimos
print(model.summary())

In [ ]:
# Utilizamos el método fit para ajustar los datos de nuestro modelo a la configuración que definimos
history = model._____(X_train, y_train, batch_size=10, epochs=10, verbose=1, validation_split=0.2)

# Utilizamos evaluate para determinar la pérdida y el accuracy de nuestro modelo
score = model._____(X_test, y_test, verbose=1)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
# Finalmente imprimimos la eficiencia y la pérdida del modelo
# época a época para ver su evolución
import matplotlib.pyplot as plt


plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

## (Reto 0)4 Paso 6 - Aplicación de una Red Convolucional (CNN)

In [ ]:
# Declaración de modelo Secuencial
model = _____()

# Declaración de las capas del modelo convolucional
embedding_layer = _____(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(_____)
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# Impresión de parámetros del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
# Utilizamos el método fit para ajustar los datos de nuestro modelo a la configuración que definimos
history = model._____(X_train, y_train, batch_size=10, epochs=10, verbose=1, validation_split=0.2)

# Utilizamos evaluate para determinar la pérdida y el accuracy de nuestro modelo
score = model._____(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
# Finalmente imprimimos la eficiencia y la pérdida del modelo
# época a época para ver su evolución
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

## (Reto 05) Paso 6.1 - Red LSTM

In [ ]:
# Declaración de modelo Secuencial
model = Sequential()

# Declaración de las capas del modelo LSTM
embedding_layer = _____(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(_____)
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# Impresión de parámetros del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
# Utilizamos el método fit para ajustar los datos de nuestro modelo a la configuración que definimos
history = model._____(X_train, y_train, batch_size=10, epochs=10, verbose=1, validation_split=0.2)

# Utilizamos evaluate para determinar la pérdida y el accuracy de nuestro modelo
score = model._____(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
# Finalmente imprimimos la eficiencia y la pérdida del modelo
# época a época para ver su evolución
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

## Paso 7 - Probador de información

In [ ]:
utterance = "Esta es una película de verdad decepcionante, aburrida y que realmente no puedes disfrutar"
#utterance = "Esta es una gran película, me encantó, es de lo mejor del año"

sequence = tokenizer.texts_to_sequences([utterance])
padded = pad_sequences(sequence, padding='post', maxlen=maxlen)

# Resultado de la evaluación
print("Resultado del modelo")
model.predict(padded)